Q2 projected gradient descent algorithm

In [1]:
from gurobipy import Model, GRB
import numpy as np
import pandas as pd

# Define the price response function coefficients for the two products
df = pd.read_csv('/Users/pratiksha/Downloads/price_response.csv')  

intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())
capacities = df['Capacity'].to_numpy()

a1, a2, b1, b2 = intercepts[0], intercepts[1], sensitivities[0], sensitivities[1]

# Define the initial prices for both products
initial_prices = np.array([0, 0])

# Define the step size and stopping criterion for the gradient descent
step_size = 0.001
stopping_criterion = 1e-6

# Define the demand functions for the two products
def demand(p, a, b):
    return a - b * p

# Define the gradient of the objective function
def gradient(p):
    return np.array([
        a1 - 2 * b1 * p[0],  # Partial derivative with respect to p1
        a2 - 2 * b2 * p[1]   # Partial derivative with respect to p2
    ])

# Initialize the prices
prices = initial_prices

# Create the Gurobi model and variables outside of the loop
m = Model('Projection')
m.setParam('OutputFlag', 0) # Suppress Gurobi output

p1 = m.addVar(lb=0, name="p1")
p2 = m.addVar(lb=0, name="p2")

m.addConstr(a1 - b1 * p1 >= 0, "DemandNonNegativityBasic")
m.addConstr(a2 - b2 * p2 >= 0, "DemandNonNegativityAdvanced")
m.addConstr(p2 - p1 >= 0.01, "PriceOrdering")

iteration_counter = 0

# List to store revenue at each iteration
revenues = []

# Start the projected gradient descent algorithm
while True:
    iteration_counter += 1  # Increment the counter
    # Compute the gradient at the current prices
    grad = gradient(prices)
    
    # Take a step in the direction of the gradient
    new_prices = prices + step_size * grad
    
    # Update the model with the new objective function
    m.setObjective((p1 - new_prices[0])*(p1 - new_prices[0]) +
                   (p2 - new_prices[1])*(p2 - new_prices[1]),
                   GRB.MINIMIZE)

    # Optimize the model
    m.optimize()
    
    # Extract the projected prices
    projected_prices = np.array([p1.X, p2.X])
    
    # Calculate and store the revenue at the current prices
    current_revenue = projected_prices[0] * (a1 - b1 * projected_prices[0]) + projected_prices[1] * (a2 - b2 * projected_prices[1])
    revenues.append(current_revenue)
    
    # Check the stopping criterion
    if np.linalg.norm(prices - projected_prices) < stopping_criterion:
        break
    
    # Update the prices
    prices = projected_prices
    if iteration_counter % 15 == 0:  # Print every 15 iterations to track progress
        print(f'Iteration {iteration_counter}: Prices = {prices}, Revenue = {current_revenue}')

# Print the final prices and revenue
print("\n")
print(f'The model ran {iteration_counter} iterations.')
print("Optimal prices found:", prices)
print("Optimal revenue:", revenues[-1])

Restricted license - for non-production use only - expires 2025-11-24
Iteration 15: Prices = [293.28898671 505.99254082], Revenue = 23400994.273436334
Iteration 30: Prices = [362.48310029 900.49865794], Revenue = 34099570.06101777
Iteration 45: Prices = [ 378.80770017 1208.08239466], Revenue = 40406756.52060907
Iteration 60: Prices = [ 382.65907635 1447.8955458 ], Revenue = 44229856.07962936
Iteration 75: Prices = [ 383.56771108 1634.8701501 ], Revenue = 46553241.22020462
Iteration 90: Prices = [ 383.78208045 1780.64823811], Revenue = 47965551.28245162
Iteration 105: Prices = [ 383.83265548 1894.30672677], Revenue = 48824067.20523352
Iteration 120: Prices = [ 383.84458738 1982.92259643], Revenue = 49345943.44097765
Iteration 135: Prices = [ 383.84740241 2052.01354113], Revenue = 49663182.583457835
Iteration 150: Prices = [ 383.84806654 2105.88152953], Revenue = 49856026.53439989
Iteration 165: Prices = [ 383.84822323 2147.88066832], Revenue = 49973252.89297947
Iteration 180: Prices = [

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_33573/2487788413.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


_____

Q2 CHATGPT quadratic optimization model

In [2]:
from gurobipy import Model, GRB

# Define the data
data = {
    'Product Line': ['Line 1', 'Line 1', 'Line 1', 'Line 2', 'Line 2', 'Line 2', 'Line 3', 'Line 3', 'Line 3'],
    'Version': ['Basic', 'Advanced', 'Premium', 'Basic', 'Advanced', 'Premium', 'Basic', 'Advanced', 'Premium'],
    'Intercept': [135234.55, 237790.24, 335675.33, 137041.38, 236846.14, 335827.02, 139414.27, 235991.95, 339313.32],
    'Sensitivity': [-45.8964, -8.22779, -7.58444, -9.03317, -4.42787, -2.62906, -2.42148, -4.00051, -2.29662],
    'Capacity': [80020.0, 89666.0, 80638.0, 86740.0, 84050.0, 86565.0, 87051.0, 85156.0, 87588.0]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Define the price response function coefficients for the two products
intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())
capacities = df['Capacity'].to_numpy()

a1, a2, b1, b2 = intercepts[0], intercepts[1], sensitivities[0], sensitivities[1]

# Define the initial prices for both products
initial_prices = np.array([0, 0])

# Define the step size and stopping criterion for the gradient descent
step_size = 0.001
stopping_criterion = 1e-6

# Define the demand functions for the two products
def demand(p, a, b):
    return a - b * p

# Define the gradient of the objective function
def gradient(p):
    return np.array([
        a1 - 2 * b1 * p[0],  # Partial derivative with respect to p1
        a2 - 2 * b2 * p[1]   # Partial derivative with respect to p2
    ])

# Initialize the prices
prices = initial_prices

# Create the Gurobi model and variables outside of the loop
m = Model('Projection')
m.setParam('OutputFlag', 0) # Suppress Gurobi output

p1 = m.addVar(lb=0, name="p1")
p2 = m.addVar(lb=0, name="p2")

m.addConstr(a1 - b1 * p1 >= 0, "DemandNonNegativityBasic")
m.addConstr(a2 - b2 * p2 >= 0, "DemandNonNegativityAdvanced")
m.addConstr(p2 - p1 >= 0.01, "PriceOrdering")

iteration_counter = 0

# List to store revenue at each iteration
revenues = []

# Start the projected gradient descent algorithm
while True:
    iteration_counter += 1  # Increment the counter
    # Compute the gradient at the current prices
    grad = gradient(prices)
    
    # Take a step in the direction of the gradient
    new_prices = prices + step_size * grad
    
    # Update the model with the new objective function
    m.setObjective((p1 - new_prices[0])*(p1 - new_prices[0]) +
                   (p2 - new_prices[1])*(p2 - new_prices[1]),
                   GRB.MINIMIZE)

    # Optimize the model
    m.optimize()
    
    # Extract the projected prices
    projected_prices = np.array([p1.X, p2.X])
    
    # Calculate and store the revenue at the current prices
    current_revenue = projected_prices[0] * (a1 - b1 * projected_prices[0]) + projected_prices[1] * (a2 - b2 * projected_prices[1])
    revenues.append(current_revenue)
    
    # Check the stopping criterion
    if np.linalg.norm(prices - projected_prices) < stopping_criterion:
        break
    
    # Update the prices
    prices = projected_prices
    if iteration_counter % 15 == 0:  # Print every 15 iterations to track progress
        print(f'Iteration {iteration_counter}: Prices = {prices}, Revenue = {current_revenue}')

# Print the final prices and revenue
print("\n")
print(f'The model ran {iteration_counter} iterations.')
print("Optimal prices found:", prices)
print("Optimal revenue:", revenues[-1])


Iteration 15: Prices = [1125.68009895 3183.89329875], Revenue = 767765149.3350751
Iteration 30: Prices = [1391.25626456 5666.27286215], Revenue = 1182526743.9291666
Iteration 45: Prices = [1453.91234226 7601.70464914], Revenue = 1431761377.8951252
Iteration 60: Prices = [1468.69448231 9110.69878417], Revenue = 1583105710.0009441
Iteration 75: Prices = [ 1472.18195986 10287.213149  ], Revenue = 1675096264.1977618
Iteration 90: Prices = [ 1473.00474327 11204.50369277], Revenue = 1731015178.0658462
Iteration 105: Prices = [ 1473.19885851 11919.68573121], Revenue = 1765007257.848269
Iteration 120: Prices = [ 1473.24465517 12477.29024547], Revenue = 1785670433.5973558
Iteration 135: Prices = [ 1473.25545975 12912.03661439], Revenue = 1798231208.4533787
Iteration 150: Prices = [ 1473.25800882 13250.99442492], Revenue = 1805866678.8128557
Iteration 165: Prices = [ 1473.25861021 13515.26896641], Revenue = 1810508144.670808
Iteration 180: Prices = [ 1473.25875209 13721.31535659], Revenue = 1813

_____

q2 chatgpt code, high chance it's wrong

In [3]:
from gurobipy import Model, GRB
import numpy as np
import pandas as pd

# Load the dataset into a pandas DataFrame (assuming it's already loaded)
df = pd.read_csv('/Users/pratiksha/Downloads/price_response.csv')  

# Define the price response function coefficients for the products
intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())

# Define the initial prices for the products
initial_prices = np.zeros(len(intercepts))

# Define the step size and stopping criterion for the gradient descent
step_size = 0.001
stopping_criterion = 1e-6

# Define the demand functions for the products
def demand(prices, intercepts, sensitivities):
    return intercepts - sensitivities * prices

# Define the gradient of the objective function
def gradient(prices, intercepts, sensitivities):
    return -2 * sensitivities * (intercepts - sensitivities * prices)

# Initialize the prices
prices = initial_prices

# Create the Gurobi model and variables outside of the loop
m = Model('Projection')
m.setParam('OutputFlag', 0) # Suppress Gurobi output

price_vars = [m.addVar(lb=0, name=f"price_{i}") for i in range(len(prices))]

# Set the initial values for the price variables
for i, price_var in enumerate(price_vars):
    price_var.start = prices[i]

# Add constraints for demand non-negativity
for i, price_var in enumerate(price_vars):
    m.addConstr(intercepts[i] - sensitivities[i] * price_var >= 0, f"DemandNonNegativity_{i}")

# Start the projected gradient descent algorithm
iteration_counter = 0
while True:
    iteration_counter += 1

    # Compute the gradient at the current prices
    grad = gradient(prices, intercepts, sensitivities)
    
    # Take a step in the direction of the gradient
    new_prices = prices + step_size * grad
    
    # Project the new prices to satisfy the non-negativity constraints
    new_prices = np.maximum(new_prices, np.zeros(len(new_prices)))

    # Update the model with the new objective function
    m.setObjective(sum((price_vars[i] - new_prices[i])*(price_vars[i] - new_prices[i]) for i in range(len(prices))),
                   GRB.MINIMIZE)

    # Optimize the model
    m.optimize()
    
    # Extract the projected prices
    projected_prices = np.array([var.X for var in price_vars])
    
    # Check the stopping criterion
    if np.linalg.norm(prices - projected_prices) < stopping_criterion:
        break
    
    # Update the prices
    prices = projected_prices

# Print the final prices and objective value
print("\n")
print(f'The model ran {iteration_counter} iterations.')
print("Optimal prices found:", prices)
print("Optimal objective value:", m.objVal)




The model ran 1 iterations.
Optimal prices found: [0. 0. 0. 0. 0. 0. 0. 0. 0.]
Optimal objective value: 0.0
